# Withdraw Liquidity
A position $\alpha$ with $i_\alpha = i$ and $\Delta s_\alpha$ is withdrawn/destroyed. Recall that by the sign conventions, $\Delta s_\alpha < 0$.

In [2]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.path.abspath(''), '..')))

import inspect
from model.amm.omnipool_amm import remove_risk_liquidity

### Constraints
Removing liquidity must leave prices $p_j^Q$ unchanged for all assets $j$. It must also leave $\frac{R_i}{S_i}$ unchanged. We furthermore require that the value of the assets withdrawn ($p_i^Q\Delta r_\alpha + \Delta q_\alpha$) is $\frac{2p_i^Q}{p_i^Q + p_\alpha}\sqrt{\frac{p_i^Q}{p_\alpha}}$ percent of the assets originally contributed to the pool.

Note that $\Delta s_\alpha$ corresponds to $-\frac{\Delta s_\alpha}{S_i}R_i$ currently. We can calculate the initial $i$ contributed by $\alpha$ as $R_i^\alpha = \sqrt{\frac{p_i^Q}{p_\alpha}} \frac{-\Delta s_\alpha}{S_i}R_i$. The value withdrawn by LP should thus be (denominated in $i$) $\frac{2p_i^Q}{p_i^Q + p_\alpha}\frac{p_i^Q}{p_\alpha} \frac{-\Delta s_\alpha}{S_i}p_\alpha R_i$.

### Requirements
$$
-\Delta s_\alpha \leq s_\alpha\\
\Delta s_\alpha < 0
$$

### Updating AMM state

$$
\begin{align}
\Delta B_i &= max\left(\frac{p_i^Q - p_\alpha}{p_i^Q + p_\alpha}\Delta s_\alpha, 0\right)\\
\Delta S_i &= \Delta s_\alpha + \Delta B_i\\
\Delta R_i &= \frac{R_i}{S_i} \Delta S_i \\
\Delta Q_i &= Q_i \frac{\Delta R_i}{R_i}\\
\Delta L &= \Delta R_i \frac{Q_i}{R_i} \frac{L}{Q}\\
\Delta T_i &= Q_i^+ \frac{R_U^+}{Q_U^+} - T_i\\
\Delta T &= \Delta T_i
\end{align}
$$


### Updating agent state

$$
\begin{align}
\Delta r_\alpha &= - \Delta R_i\\
\Delta q_\alpha &= - p_i^Q\left(\frac{2p_i^Q}{p_i^Q + p_\alpha} \frac{\Delta s_\alpha}{S_i}R_i + \Delta r_\alpha\right)
\end{align}
$$

### Updating NFT in case of partial withdraw
If $s_\alpha + \Delta s_\alpha > 0$, the LP is only partially withdrawing their liquidity.

Tracking the quantity of assets initially deposited is not required, but may be desirable for other reasons. Let us denote this quantity $r_\alpha$.
Upon *partial* liquidity withdrawal, we recalculate $r_\alpha$ as though the position is being split into two positions, and one entire position is being withdrawn.
$$
\Delta r_\alpha = r_\alpha \frac{\Delta s_\alpha}{s_\alpha}
$$

In [3]:
print(inspect.getsource(remove_risk_liquidity))

def remove_risk_liquidity(
        old_state: dict,
        old_agents: dict,
        LP_id: string,
        delta_S: float,
        i: int
) -> tuple:
    """Compute new state after liquidity removal"""
    assert delta_S <= 0, "delta_S cannot be positive: " + str(delta_S)
    assert i >= 0, "invalid value for i: " + str(i)

    new_state = copy.deepcopy(old_state)
    new_agents = copy.deepcopy(old_agents)

    if delta_S == 0:
        return new_state, new_agents

    piq = price_i(old_state, i)
    p0 = new_agents[LP_id]['p'][i]
    mult = (piq - p0) / (piq + p0)

    # Share update
    delta_B = max(mult * delta_S, 0)
    new_state['B'][i] += delta_B
    new_state['S'][i] += delta_S + delta_B
    new_agents[LP_id]['s'][i] += delta_S

    # Token amounts update
    delta_R = old_state['R'][i] * max((delta_S + delta_B) / old_state['S'][i], -1)
    new_state['R'][i] += delta_R
    new_agents[LP_id]['r'][i] -= delta_R
    if piq >= p0:  # prevents rounding errors
        new_agents[LP